## Project Expat Assistance

Starting with the Web Framework with Flask

In [40]:
#installing the flask

#remove in case flask is not installed
#pip uninstall flask 

#in case upgrade needed
#!pip install --upgrade flask

#flask --version

## Running Example code with Flask

In [3]:
from flask import Flask, render_template  #importing libraries
#from jinja2 import pass_context

app = Flask(__name__)    #__name__ is communicating with Flask function for user's
                         #current directory specification

#routing in FLask
@app.route("/")          #Calling flask to route to the end of a website eg"xcvb.com/" to execute a code
def Landing_Page():         #function Landing Page at/
    try:
        return render_template("home.html")   #returning Hello World at the end (when someone visits the website (full usl ending with /))
    except Exception as e:
        return str(e)


@app.route("/home")
def Home_Page():
     return "Help us to Help You!"

@app.route("/home/apply")
def Application_Page():
     return "Choose your way"

@app.route("/home/apply/trc")
def Apply_TRC():
     return "Submit your response here"

app.run(host='localhost', port=8080) #running app which is Flask at localhost at port 8080



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Dec/2022 11:11:23] "GET / HTTP/1.1" 200 -
